<a href="https://colab.research.google.com/github/Lilcob/test_colab/blob/main/0213_ourmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import PIL  # python 의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰때 사용
import os # os, path 관련 조작, 변경 , 파일 삭제 등
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 우리만의 데이터, 모델 만들어 보기
# 두가지 카테고리 분류 (데이터 기획은 여러분 들이)
# https://chromewebstore.google.com/detail/zzllrr-imager-geek/gfjhimhkjmipphnaminnnnjpnlneeplk
# 이미저 긱 설치
# 이미지 검색 -> 우측 상단 아이콘 버튼 클릭 GO -> 필요 없는 이미지 클릭해서 제거
# Rename 버튼 클릭후, -> prefix 명명규칙 수정 -> save 버튼 눌러서 저장
# 각각 카테고리에 이미지 500장
# 두 폴더 압축하기
# 압축한 폴더를 드라이브에 압축한 파일 업로드
# google drive 마운트 , colab 옆 폴더 마크 , 상단 google drive mount 버튼 클릭
# 업로드한 압축 파일이 있는지 확인.

In [3]:
# /content/drive/MyDrive/ourdatset.zip
!mkdir dataset

In [ ]:
!unzip /content/drive/MyDrive/ourdatset.zip -d dataset

In [5]:

dataset_folder_path = '/content/dataset'
category_cnt = len(os.listdir(dataset_folder_path))
print(os.listdir(dataset_folder_path))

['PNEUMONIA', 'NORMAL']


 **glob.glob(dataset_folder_path+'/*/*')**
*   파일의 경로+/* 을 입력하면 해당 폴더 하위
*   경로 상에 있는 모든 파일을 리스트로 리턴



```
ex) dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴
```



In [ ]:
x = []
y = []

for label,folder in enumerate(os.listdir(dataset_folder_path)):

    # print(dataset_folder_path +'/'+ folder) 좋은 코드가 아님
    # os.path.join(dataset_folder_path , folder)) # 유지 보수 좋은 코드

    folder_full_path = os.path.join(dataset_folder_path , folder)
    all_files = glob.glob(folder_full_path + '/*.jpeg')
    all_files_jpg = glob.glob(folder_full_path + '/*.jpg')
    all_files_png = glob.glob(folder_full_path + '/*.png')
    all_files.extend(all_files_jpg)
    all_files.extend(all_files_png)

    for idx,file_full_path in enumerate(all_files):
        image = PIL.Image.open(file_full_path)
        image = image.resize((64,64))
        image = image.convert('RGB')
        data = np.asarray(image)
        x.append(data)
        y.append(label)
        if idx % 30 == 0:
            print(idx,'/',len(all_files))

In [7]:
x = np.array(x)
y = np.array(y)
print('카테고리 갯수',np.bincount(y))
print('전처리 확인',x.shape)

카테고리 갯수 [3875 1341]
전처리 확인 (5216, 64, 64, 3)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
# 개발 팁 , 전처리 시간이 너무 오래 걸린다면(데이터 늘어 나서)
# xy = (x_train, x_test, y_train, y_test)
# np.save('/content/drive/MyDrive/preprocessing_data.npy',xy)
# x_train,x_test,y_train,y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy')

In [10]:
x_train = x_train / 255.
x_test = x_test / 255.

In [11]:
# 구글넷 실습(오전에 진행한)
# 레즈넷 실습(다 짜여진 코드) -> fine tuning

In [ ]:
# 개선의 여지 해상도 수정 ( 전처리 도 수정 해야 함)
input_layer = layers.Input(shape = (64,64,3), name='start_layer')

# 개선의 여지 conv2d 수정 성능이 매우 잘 나온다면 -모델 다이어트
# 성능이 안나온다면 channel 수, 레이어 추가
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

final_concat_layer = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)

# 개선의 여지 -  Dense 층 추가
output_layer = layers.Dense(1 , activation = 'sigmoid')(flat_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

In [17]:
#개선 여지 hyper parameter tuning
# ex ) tf.keras.optimizers.Adam()
google_net_model.compile(loss='binary_crossentropy',
                         optimizer='adam', metrics=['accuracy'])

In [18]:
check_point_path = '/content/drive/MyDrive/ourmodel_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                   monitor='val_loss',
                                   save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
google_net_model.fit(x_train,y_train, batch_size=32,
                     epochs=20,
                     callbacks=[mc,es],
                     validation_data=(x_test,y_test))

In [6]:
## our model inference code
# inference만 진행 할 시, 가장 위 import library 실행 -> 모델 빌드 하는 부분 실행
google_net_model.load_weights('/content/drive/MyDrive/ourmodel_checkpoint')

In [ ]:
from google.colab import files
f = files.upload()

In [ ]:
file_name = list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64,64))
test_img = test_img.convert('RGB')
data = np.asarray(test_img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape

In [44]:
result = google_net_model.predict(test_image_tensor)
result[0][0]
if result[0][0] > 0.5:
    print(f'{result[0][0] * 100} % 확률로 정상 폐로 판단이 됩니다.')
else:
    print(f'{ (1 - result[0][0]) * 100} % 폐 질환이 예측 됩니다.')

1/1 [==============================] - 0s 54ms/step
99.97873264801456 % 폐 질환이 예측 됩니다.
